In [ ]:
import pandas as pd
import numpy as np

In [ ]:
business = pd.read_json("yelp_academic_dataset_business.json", lines=True)

Sneak peek into 2 first rows to realise what we've got.

In [ ]:
business.head(2)

Create 2 bool arrays to finaly get only Austing AND Texas entries.

In [ ]:
austin_tx_bool = np.array(business.city == 'Austin') * np.array(business.state == 'TX')
austin_tx = business[austin_tx_bool]

In [ ]:
# austin_tx.to_json('austin_tx.json')

Drop unimportant columns.

In [ ]:
# austin_tx = austin_tx.drop(['city', 'state', 'business_id', 'address'], axis=1)
austin_tx = austin_tx.drop(['city', 'state', 'business_id'], axis=1)

We have a lot of nan values, let's see how many.

In [ ]:
austin_tx.isna().sum()

Let's drop the nans.

In [ ]:
dropped = austin_tx.shape[0]
austin_tx.dropna(inplace=True)
dropped -= austin_tx.shape[0]
print("Dropped:", dropped, "rows with nan values.") #5070

In [ ]:
# austin_tx.to_json('austin_tx_nona.json')

Let's peek to first 2 rows to see what we've got.

In [ ]:
austin_tx.head(2)

Add a column stating distance in... kinda minutes in GPS sense I guess... Doesn't really matter - what matters is we can compare distances from the centre now.

In [ ]:
lat_lon_df = austin_tx.loc[:,'latitude':'longitude']
number_of_rows = lat_lon_df.shape[0]

gps_based_distance = np.zeros(shape=number_of_rows)

for row in np.arange(number_of_rows):

    lat_lon = np.array([lat_lon_df.iloc[row][0], lat_lon_df.iloc[row][1]]) * 60
    capitol_lat_lon = np.array([30.274773446583634, -97.74038126660496]) * 60
    gps_based_distance[row] = np.linalg.norm(lat_lon - capitol_lat_lon)

austin_tx['dist_from_center'] = gps_based_distance

Clear address column from strange informations.

In [ ]:
def fix_address(address):
    return (address.split(sep=', '))[0]

In [ ]:
austin_tx['address'] = austin_tx['address'].apply(fix_address)

In [ ]:
austin_tx

Work In progress - function to get all the distances from a given location to given addressess.

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import distance

def get_distances(dataframe, addresses_column_name, given_location, postifx="", inplace=False): # TODO: add mechanics for inplace=True
    """
    dataframe - pandas.DataFrame object
    addresses_column_name - name of the column in which we can find addresses
    given_location - tuple with latitude and longitude of a location from which we're counting the distance

    postfix - what should I add to every address to make sure it's what we need. example: city/country

    returned value - pd.Series distances in kilometers
    """
    geolocator = Nominatim(user_agent="Strive School YMVC Project")

    number_of_rows = dataframe.shape[0]
    addresses_df = dataframe[addresses_column_name]

    gps_addresses = np.zeros(shape=number_of_rows)

    for row_nr in np.arange(0, 50):  ## TODO: fix this for the real rannge, this is a testing one
        dist = np.nan
        try:
            loc = geolocator.geocode(addresses_df.iloc[row_nr] + postifx)
            dist = distance(given_location, (loc.latitude, loc.longitude)).kilometers
        except Exception as e:
            print(e)
        gps_addresses[row_nr] = dist
        
    return pd.Series(gps_addresses, name="distance")

In [ ]:
from_here = (30.274773446583634, -97.74038126660496)
x = get_distances(austin_tx, 'address', from_here, postifx=" Austin Texas")

In [ ]:
x[15:18]

Useful function to help with selecting only rows which contain a chosen word in a given column.

In [ ]:
def contains(column, word):
    """
    Usage:
    Pass a column and a word u want to find in the cells, to get a np.array of type bool.
    Afterwards u can pass it as an argument of [] selection mechanism. Examples:

    austin_tx[ contains(austin_tx.categories, "Shopping") ]
    austin_tx[ contains(austin_tx.categories, "Shopping") | contains(austin_tx.categories, "Hotels") ]      # and
    austin_tx[ contains(austin_tx.categories, "Shopping") & contains(austin_tx.categories, "Watches") ]     # or

    Works for all columns that return strings. hours and attributes too.
    """
    return np.array([(word in vals) for vals in column])

Chech for ALL existing cathegories. (Additionaly it checks how many of each one are there.)

In [ ]:
categories_counts = {}
for things in austin_tx.categories.unique():
    things = things.replace(', &', ' &')
    for thing in things.split(sep=", "):
        if thing in categories_counts.keys():
            categories_counts[thing] += 1
        else:
            categories_counts[thing] = 1